In [24]:
import os
import librosa
from pydub import AudioSegment
import tensorflow as tf
from keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.models import Sequential
import Paths
from SampleModel import SampleModel
from keras import models, layers
import time

directory = os.listdir(Paths.IN_DIRECTORY)

EDGE_SIZE = 50


def define_model(len_ts,
                 hidden_neurons = 10,
                 nfeature=1,
                 batch_size=None,
                 stateful=False):
    in_out_neurons = 1

    inp = layers.Input(batch_shape= (batch_size, len_ts, nfeature),
                       name="input")

    rnn = layers.LSTM(hidden_neurons,
                    return_sequences=True,
                    stateful=stateful,
                    name="RNN")(inp)

    dens = layers.TimeDistributed(layers.Dense(in_out_neurons,name="dense"))(rnn)
    model = models.Model(inputs=[inp],outputs=[dens])

    model.compile(loss="mean_squared_error",
                  sample_weight_mode="temporal",
                  optimizer="rmsprop")
    return(model,(inp,rnn,dens))


def load_sample_edges(path: str):
    start_samples = []
    end_samples = []
    files = os.listdir(path)
    files = sorted(files, key=lambda x: int(x.split('_')[-1].split(".")[0]))
    print(files[:11])
    i = 0
    #sequence_length = []
    sampleModel = []
    for file_name in files:
        if 'wav' in file_name:
            complete_path = path + file_name
            sample = AudioSegment.from_wav(complete_path)
            sample_array = sample.get_array_of_samples()
            start_array = sample_array[:EDGE_SIZE]
            start_samples.append(start_array)
            end_array = sample_array[-EDGE_SIZE:]
            end_samples.append(end_array)
            model = SampleModel(name=file_name, start=start_array, end=end_array)
            sampleModel.append(model)
            #sequence_length.append(len(model))
            i += 1

    end_samples = end_samples[:-1]
    start_samples = start_samples[1:]

    return end_samples, start_samples, sampleModel


end_samples, start_samples, sampleModel = load_sample_edges(Paths.IN_DIRECTORY)



source_samples = []
target_samples = []
source_values = set()
target_values = set()

i = 0

while i < 9:
    #print(end_samples[i])
    #print(start_samples[i])
    i += 1


i = 0

while i < len(end_samples):
    source_samples.append(end_samples[i])
    target_samples.append(start_samples[i])
    for value in end_samples[i]:
        if value not in source_values:
            source_values.add(value)
    for value in start_samples[i]:
        if value not in target_values:
            target_values.add(value)
    i += 1

source_values_sorted = sorted(list(source_values))
target_values_sorted = sorted(list(target_values))

#source_samples = tf.convert_to_tensor(source_samples)
#target_samples = tf.convert_to_tensor(target_samples)

input_data = tf.reshape(source_samples, shape=(len(source_samples), EDGE_SIZE, 1))
output_data = tf.reshape(target_samples, shape=(len(source_samples), EDGE_SIZE, 1))

'''


# Removing the last element using slicing
lastElementIndex = len(array)-1
in_array = array[:lastElementIndex]
out_array = array[1:]


#Create the Input and Output to tensors
input_tensor = tf.convert_to_tensor(in_array)
output_tensor = tf.convert_to_tensor(out_array)



timestep = sequence_length[0]

input_data = tf.reshape(input_tensor, shape=(batch_size, timestep, 1))
output_data = tf.reshape(output_tensor, shape=(batch_size, timestep, 1))



# Test mit Conv1D Layer, MaxPooling1D Layer, Aktivierungsfunktion Sigmoid
# Mögliche Anpassungen: Anzahl der Samples, Aktivierungsfunktion

# define model
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3,strides=1, padding="causal", activation="sigmoid",input_shape=[None, 1]))
model.add(MaxPooling1D(pool_size=2, strides=1, padding='same'))
model.add(LSTM(100, activation='sigmoid', return_sequences=True, input_shape=(EDGE_SIZE, 1)))
model.add(Dense(1))
'''


model = Sequential()
model.add(LSTM(10, activation='relu', return_sequences=True, input_shape=(EDGE_SIZE, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
#model.summary()

# fit model
start = time.time()
model.fit(input_data, output_data, epochs=20000, verbose=1)
end = time.time()
print("Time Took :{:3.2f} min".format( (end-start)/60 ))

['Tea K Pea - nauticals_1.wav', 'Tea K Pea - nauticals_2.wav', 'Tea K Pea - nauticals_3.wav', 'Tea K Pea - nauticals_4.wav', 'Tea K Pea - nauticals_5.wav', 'Tea K Pea - nauticals_6.wav', 'Tea K Pea - nauticals_7.wav', 'Tea K Pea - nauticals_8.wav', 'Tea K Pea - nauticals_9.wav', 'Tea K Pea - nauticals_10.wav']
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 50, 1)]           0         
                                                                 
 RNN (LSTM)                  (None, 50, 64)            16896     
                                                                 
 time_distributed (TimeDistr  (None, 50, 1)            65        
 ibuted)                                                         
                                                                 
Total params: 16,961
Trainable params: 16,961
Non-trainable params: 0
_________

"\nmodel = Sequential()\nmodel.add(LSTM(10, activation='relu', return_sequences=True, input_shape=(EDGE_SIZE, 1)))\nmodel.add(Dense(1))\nmodel.compile(optimizer='adam', loss='mse')\n#model.summary()\n\n# fit model\nmodel.fit(input_data, output_data, epochs=20000, verbose=1)\n"

In [16]:
# demonstrate prediction
x_input = source_samples[4]
x_input = tf.reshape(x_input, (1, EDGE_SIZE, 1))
yhat = model.predict(x_input, verbose=2)
print(yhat)

1/1 - 0s - 138ms/epoch - 138ms/step
[[[ 688.46576]
  [ 839.8293 ]
  [ 909.2725 ]
  [ 937.69855]
  [ 948.60583]
  [ 952.67413]
  [ 954.182  ]
  [ 954.7379 ]
  [ 954.94257]
  [ 955.0178 ]
  [ 955.0723 ]
  [ 955.0825 ]
  [ 955.1123 ]
  [ 955.11365]
  [ 955.11414]
  [ 955.11444]
  [ 955.1145 ]
  [ 955.1145 ]
  [ 955.1145 ]
  [ 955.1145 ]
  [ 955.1145 ]
  [ 955.1087 ]
  [ 955.1087 ]
  [ 953.3539 ]
  [ 953.19586]
  [ 953.0825 ]
  [ 953.0143 ]
  [ 946.3453 ]
  [ 916.7831 ]
  [ 890.71765]
  [ 874.0858 ]
  [ 863.631  ]
  [ 783.1165 ]
  [ 643.125  ]
  [ 372.41098]
  [-232.65327]
  [ 549.974  ]
  [-342.99173]
  [ 598.2129 ]
  [-279.24628]
  [ 603.42145]
  [-248.40453]
  [ 604.6853 ]
  [-237.78093]
  [ 604.6423 ]
  [-236.81142]
  [ 604.2097 ]
  [-239.94588]
  [ 603.7134 ]
  [-325.84625]]]


In [21]:
print(source_samples[5])

array('h', [1653, -12, 1735, -115, 1678, -221, 1699, -255, 1841, -273, 1912, -393, 1860, -617, 1869, -848, 2021, -1057, 2138, -1300, 2103, -1568, 2060, -1759, 2148, -1845, 2231, -1933, 2131, -2083, 1954, -2222, 1913, -2303, 1961, -2419, 1881, -2636, 1690, -2839, 1614, -2889, 1670, -2830, 1618, -2795, 1388, -2788, 1195, -2717])


In [23]:
print(len(source_samples))

9
